In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from itertools import product

from models.nf_linears import LinearModels
from models.nf_mlp import MLPModels
from models.nf_transformer import TransformerModels

from datetime import date
import os

today = date.today()
data_path='./data/processed/' 

result_path=f'./results/data/{today}_NFModels'
os.makedirs(result_path, exist_ok=True)

granularity = ['daily']
features = ['event', 'fatalities']
ds_series = ["year_month", "event_date"]
seq_length = {'daily':28, 'monthly':12}



for gra, feature in product(granularity, features):
    filename=f"ts_pvtw_{gra}_{feature}.csv"
    df = pd.read_csv(f"{data_path}{filename}")  

    ds, target = df.columns
    df[ds] = pd.to_datetime(df[ds]) 
    df.insert(0, "unique_id", 1)
    df.columns = ['unique_id', 'ds', 'y']

    output_length = seq_length[gra]
    Y_train_df, Y_test_df = train_test_split(df, test_size=0.15, shuffle=False)  # 15% as testing, Y_train includes val (15%)
  
    
    print(f"Data Set Size:{df.shape[0]}")
    print(f"Testing Size:{Y_test_df.shape[0]}")

    runner_Tr = TransformerModels(Y_train_df, Y_test_df, output_length, gra)
    tr_metrics, Y_predicts_Tr = runner_Tr.runAdvTransformers()
   
           
    runner_Linear = LinearModels(Y_train_df, Y_test_df, output_length, gra)
    dlinear_metrics, Y_predicts_dlinear = runner_Linear.runDLinear()    
    nlinear_metrics, Y_predicts_linear = runner_Linear.runLinear()  
    
    runner_mlp = MLPModels(Y_train_df, Y_test_df, output_length, gra)
    mlp_metrics, Y_predicts_mlp = runner_mlp.runMLPs()
    
        
    metrics = pd.concat([nlinear_metrics, dlinear_metrics,mlp_metrics, tr_metrics], ignore_index = True)
    fname = f"{result_path}/{gra}_{feature}_metrics.csv"
    metrics.round(6).to_csv(fname, index = False)

    results_df = pd.concat([Y_predicts_linear, Y_predicts_dlinear,Y_predicts_mlp, Y_predicts_Tr], axis = 1)
    fname = f"{result_path}/{gra}_{feature}_results.csv"
    results_df.round(6).to_csv(fname, index = False) 
    

 
    
